In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
sns.set()
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv(r"C:\Users\hp\Downloads\creditcard.csv\creditcard.csv")
df = data.copy() # To keep the data as backup
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hp\\Downloads\\creditcard.csv\\creditcard.csv'

In [ ]:
df.shape


In [ ]:
df.isnull().sum()

In [3]:
df.dtypes

NameError: name 'df' is not defined

In [ ]:
df.Time.tail(15)

In [ ]:
df.describe()

In [ ]:
df.Class.value_counts()

In [ ]:
sns.countplot(x=df.Class, hue=df.Class)

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df.Amount)

In [ ]:
df['Amount-Bins'] = ''


In [ ]:
def make_bins(predictor, size=50):
    '''
    Takes the predictor (a series or a dataframe of single predictor) and size of bins
    Returns bins and bin labels
    '''
    bins = np.linspace(predictor.min(), predictor.max(), num=size)

    bin_labels = []

    # Index of the final element in bins list
    bins_last_index = bins.shape[0] - 1

    for id, val in enumerate(bins):
        if id == bins_last_index:
            continue
        val_to_put = str(int(bins[id])) + ' to ' + str(int(bins[id + 1]))
        bin_labels.append(val_to_put)
    
    return bins, bin_labels

In [ ]:
bins, bin_labels = make_bins(df.Amount, size=10)

In [ ]:
df['Amount-Bins'] = pd.cut(df.Amount, bins=bins,
                           labels=bin_labels, include_lowest=True)
df['Amount-Bins'].head().to_frame()

In [ ]:
df['Amount-Bins'].value_counts()

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(x='Amount-Bins', data=df)
plt.xticks(rotation=45)

In [ ]:

plt.figure(figsize=(15, 10))
sns.countplot(x='Amount-Bins', data=df[~(df['Amount-Bins'] == '0 to 2854')])
plt.xticks(rotation=45)

In [ ]:
df_encoded = pd.get_dummies(data=df, columns=['Amount-Bins'])
df = df_encoded.copy()

In [ ]:

df.head()

In [ ]:
X = df.drop(labels='Class', axis=1)
Y = df['Class']

X.shape, Y.shape

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(
    X, Y, random_state=42, test_size=0.3, shuffle=True)

print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()

In [ ]:
lr_model.fit(xtrain, ytrain)

In [ ]:
lr_pred_train = lr_model.predict(xtrain)
lr_pred_test = lr_model.predict(xtest)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix


In [ ]:
tn, fp, fn, tp = confusion_matrix(ytest, lr_pred_test).ravel()
conf_matrix = pd.DataFrame(
    {
        'Predicted Fraud': [tp, fp],
        'Predicted Not Fraud': [fn, tn]
    }, index=['Fraud', 'Not Fraud'])
conf_matrix

In [ ]:
sns.heatmap(conf_matrix, annot=True)


In [ ]:
lr_accuracy = accuracy_score(ytest, lr_pred_test)
lr_accuracy

In [ ]:
lr_precision = precision_score(ytest, lr_pred_test)
lr_precision

In [ ]:
lr_recall = recall_score(ytest, lr_pred_test)
lr_recall

In [ ]:
lr_recall_train = recall_score(ytrain, lr_pred_train)
lr_recall_train

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
lr_f1 = f1_score(ytest, lr_pred_test)
lr_f1

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
print(classification_report(ytest, lr_pred_test))

In [ ]:
lr_pred_test_prob = lr_model.predict_proba(xtest)[:, 1]


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
fpr, tpr, threshold = roc_curve(ytest, lr_pred_test_prob)

In [ ]:
lr_auc = roc_auc_score(ytest, lr_pred_test_prob)
lr_auc

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.figure(figsize=(8, 6))
    plt.title('ROC Curve', fontsize=15)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.xticks(np.arange(0, 1, 0.05), rotation=90)
    plt.xlabel('False Positive Rates', fontsize=15)
    plt.ylabel('True Positive Rates', fontsize=15)
    plt.legend(loc='best')
    
    plt.show()

In [ ]:
plot_roc_curve(fpr=fpr, tpr=tpr, label="AUC = %.3f" % lr_auc)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=2)
xtrain_poly = poly.fit_transform(xtrain)
xtest_poly = poly.fit_transform(xtest)

# Training the model
model = LogisticRegression()
model.fit(xtrain_poly, ytrain)

# Getting the probabilities
train_prob = model.predict_proba(xtrain_poly)[:, 1]
test_prob = model.predict_proba(xtest_poly)[:, 1]

# Computing the ROC Score
roc_auc_score(ytrain, train_prob), roc_auc_score(ytest, test_prob)

In [ ]:
fpr_poly, tpr_poly, threshold_poly = roc_curve(ytest, test_prob)

In [ ]:
plot_roc_curve(fpr=fpr_poly, tpr=tpr_poly, label='AUC = %.3f' %  roc_auc_score(ytest, test_prob))


In [ ]:
recall_score(ytest, model.predict(xtest_poly))


In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
mms = MinMaxScaler()


In [ ]:
X.head()


In [ ]:
X_scaled = mms.fit_transform(X)


In [ ]:
X_scaled = pd.DataFrame(data=X_scaled, columns=X.columns)
X_scaled.head()

In [ ]:
xtrainS, xtestS, ytrainS, ytestS = train_test_split(
    X_scaled, Y, random_state=42, test_size=0.30, shuffle=True)

In [ ]:
print(xtrainS.shape, ytrainS.shape)
print(xtestS.shape, ytestS.shape)

In [ ]:
from sklearn.svm import SVC


In [ ]:
svc_model = SVC(kernel='linear', probability=True)


In [ ]:
svc_model.fit(xtrainS, ytrainS)


In [ ]:
svc_pred = svc_model.predict(xtestS)

In [ ]:
svc_recall = recall_score(ytestS, svc_pred)


In [ ]:
svc_recall

In [ ]:
svc_pred_prob = svc_model.predict_proba(xtestS)[:, 1]


In [ ]:
svc_auc = roc_auc_score(ytestS, svc_pred_prob)

# Now, let's get the fpr and tpr
fpr, tpr, threshold = roc_curve(ytestS, svc_pred_prob)

# Now, let's draw the curve
plot_roc_curve(fpr, tpr, 'AUC: %.3f' % svc_auc)

In [ ]:
# For Kernel = rbf
tuned_rbf = {'kernel': ['rbf'], 'gamma': [
    1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}

# For kernel = sigmoid
tuned_sigmoid = {'kernel': ['sigmoid'], 'gamma': [
    1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}

# For kernel = linear
tuned_linear = {'kernel': ['linear'], 'C': [
    0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
rs_rbf = RandomizedSearchCV(estimator=SVC(probability=True), 
        param_distributions=tuned_rbf, n_iter=500, n_jobs=4, scoring='roc_auc')

rs_sigmoid = RandomizedSearchCV(estimator=SVC(probability=True), 
        param_distributions=tuned_sigmoid, n_iter=500, n_jobs=4, scoring='roc_auc')

rs_linear = RandomizedSearchCV(estimator=SVC(probability=True), 
        param_distributions=tuned_linear, n_iter=500, n_jobs=4, scoring='roc_auc')

In [ ]:
rs_rbf.fit(xtrainS, ytrainS)


In [ ]:
rs_rbf.best_estimator_


In [ ]:
svc_rbf_best_est = rs_rbf.best_estimator_


In [ ]:
svc_rbf_best_est.fit(xtrainS, ytrainS)


In [ ]:
svc_rbf_best_est_pred = svc_rbf_best_est.predict(xtestS)


In [ ]:
svc_rbf_best_est_pred_proba = svc_rbf_best_est.predict_proba(xtestS)[:, 1]


In [ ]:
svc_rbf_auc = roc_auc_score(ytestS, svc_rbf_best_est_pred_proba)


In [ ]:
svc_rbf_recall = recall_score(ytestS, svc_rbf_best_est_pred)
svc_rbf_recall

In [ ]:

fpr, tpr, threshold = roc_curve(ytestS, svc_rbf_best_est_pred_proba)
plot_roc_curve(fpr, tpr, 'AUC = %.3f' % svc_rbf_auc)

In [ ]:
rs_sigmoid.fit(xtrainS, ytrainS)


In [ ]:
svc_sigmoid = rs_sigmoid.best_estimator_


In [ ]:
svc_sigmoid.fit(xtrainS, ytrainS)


In [ ]:
svc_sigmoid_pred = svc_sigmoid.predict(xtestS)
svc_sigmoid_pred_proba = svc_sigmoid.predict_proba(xtestS)[:, 1]

In [ ]:
svc_sigmoid_auc = roc_auc_score(ytestS, svc_sigmoid_pred_proba)
svc_sigmoid_auc

In [ ]:
svc_sigmoid_recall = recall_score(ytestS, svc_sigmoid_pred)
svc_sigmoid_recall

In [ ]:
fpr, tpr, threshold = roc_curve(ytestS, svc_sigmoid_pred_proba)
plot_roc_curve(fpr, tpr, 'AUC = %.3f' % svc_sigmoid_auc)

In [ ]:
rs_linear.fit(xtrainS, ytrainS)


In [ ]:
svc_linear = rs_linear.best_estimator_


In [ ]:
svc_linear.fit(xtrainS, ytrainS)


In [ ]:
svc_linear_pred = svc_linear.predict(xtestS)
svc_linear_pred_proba = svc_linear.predict_proba(xtestS)[:, 1]

In [ ]:
svc_linear_auc = roc_auc_score(ytestS, svc_linear_pred_proba)

fpr, tpr, threshold = roc_curve(ytestS, svc_linear_pred_proba)
plot_roc_curve(fpr, tpr, 'AUC = %.3f' % svc_linear_auc)

In [ ]:
svc_linear_recall = recall_score(ytestS, svc_linear_pred)
svc_linear_recall

In [ ]:

from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()

In [ ]:
nb.fit(xtrain, ytrain)


In [ ]:
nb_pred = nb.predict(xtest)
nb_pred_proba = nb.predict_proba(xtest)[:, 1]

In [ ]:
nb_auc = roc_auc_score(ytest, nb_pred)


In [ ]:
fpr, tpr, threshold = roc_curve(ytestS, nb_pred_proba)
plot_roc_curve(fpr, tpr, 'AUC = %.3f' % nb_auc)

In [ ]:

nb_recall = recall_score(ytest, nb_pred)
nb_recall